In [29]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from PIL import Image
import itertools

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [31]:
# Step 2: Loading images and labels into arrays
def load_images_and_labels(dataset_path, img_size=(224, 224)):
    categories = ['benign', 'malignant']
    data = []
    labels = []

    for category in categories:
        folder_path = os.path.join(dataset_path, category)
        class_label = category
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            try:
                img = Image.open(img_path).resize(img_size).convert('RGB')
                data.append(np.array(img))
                labels.append(class_label)
            except Exception as e:
                continue
    return np.array(data), np.array(labels)

# Step 3: Categorical Labels
# Load data
data_path = r"C:\Users\LLR User\Desktop\Coding\code\skin-cancer\Dataset"  # Replace with your actual dataset folder path
data, labels = load_images_and_labels(data_path)

In [32]:
# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

In [33]:
# Step 4: Normalization
data = data / 255.0  # Normalize pixel values

# Step 5: Train and Test Split
X_train, X_test, y_train, y_test = train_test_split(data, labels_categorical, test_size=0.2, random_state=42, stratify=labels_encoded)


In [34]:
# Step 6: Model Building (Basic CNN)
def build_cnn_model(input_shape=(224, 224, 3)):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_cnn_model()


c:\Users\LLR User\miniconda3\envs\tf-env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [35]:
# Step 7: Cross-validating model using K-Fold (optional but shown)
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
accuracies = []

for train_idx, val_idx in kfold.split(X_train, np.argmax(y_train, axis=1)):
    model = build_cnn_model()
    model.fit(X_train[train_idx], y_train[train_idx], epochs=5, verbose=1, validation_data=(X_train[val_idx], y_train[val_idx]))
    scores = model.evaluate(X_train[val_idx], y_train[val_idx], verbose=0)
    accuracies.append(scores[1])

c:\Users\LLR User\miniconda3\envs\tf-env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 30s 772ms/step - accuracy: 0.5250 - loss: 0.7111 - val_accuracy: 0.6128 - val_loss: 0.5948
Epoch 2/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 25s 687ms/step - accuracy: 0.6790 - loss: 0.5834 - val_accuracy: 0.7744 - val_loss: 0.5055
Epoch 3/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 25s 692ms/step - accuracy: 0.7568 - loss: 0.5223 - val_accuracy: 0.7531 - val_loss: 0.4826
Epoch 4/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 25s 704ms/step - accuracy: 0.7443 - loss: 0.5129 - val_accuracy: 0.7762 - val_loss: 0.4672
Epoch 5/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 28s 764ms/step - accuracy: 0.7859 - loss: 0.4596 - val_accuracy: 0.7229 - val_loss: 0.5120
Epoch 1/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.5358 - loss: 0.7325 - val_accuracy: 0.7367 - val_loss: 0.6210
Epoch 2/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 30s 819ms/step - accuracy: 0.6269 - loss: 0.6259 - val_accuracy: 0.6673 - val_loss: 0.5546
Epoch 3/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 27s 761ms/step - accuracy: 0.7009 - loss: 0.5362 - val_accuracy: 0.779

In [36]:
print("Cross-Validation Accuracies:", accuracies)
print("Mean Accuracy:", np.mean(accuracies))

Cross-Validation Accuracies: [0.7229129672050476, 0.7953736782073975, 0.7864768505096436]
Mean Accuracy: 0.7682544986406962


In [37]:
# Step 8: Testing model
model = build_cnn_model()
model.fit(X_train, y_train, epochs=10, validation_split=0.1, verbose=1)

Epoch 1/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 37s 711ms/step - accuracy: 0.5792 - loss: 0.6880 - val_accuracy: 0.7633 - val_loss: 0.5480
Epoch 2/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 37s 755ms/step - accuracy: 0.7157 - loss: 0.5530 - val_accuracy: 0.7929 - val_loss: 0.4500
Epoch 3/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 35s 725ms/step - accuracy: 0.7260 - loss: 0.5196 - val_accuracy: 0.8284 - val_loss: 0.4353
Epoch 4/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 33s 688ms/step - accuracy: 0.7712 - loss: 0.4605 - val_accuracy: 0.7929 - val_loss: 0.4466
Epoch 5/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 33s 693ms/step - accuracy: 0.7578 - loss: 0.4804 - val_accuracy: 0.8225 - val_loss: 0.4228
Epoch 6/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 34s 706ms/step - accuracy: 0.7835 - loss: 0.4638 - val_accuracy: 0.8047 - val_loss: 0.4036
Epoch 7/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 34s 701ms/step - accuracy: 0.7726 - loss: 0.4521 - val_accuracy: 0.8047 - val_loss: 0.4109
Epoch 8/10
48/48 ━━━━━━━━━━━━━━━━━━━━ 32s 662ms/step - accuracy: 0.7885 - loss: 0.4264 - val_accu

In [38]:
# Evaluate on test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 149ms/step


In [39]:
# Accuracy & classification report
acc = accuracy_score(y_true, y_pred_classes)
print("Test Accuracy:", acc)
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred_classes))
print("Classification Report:\n", classification_report(y_true, y_pred_classes))

Test Accuracy: 0.7630331753554502
Confusion Matrix:
 [[173  58]
 [ 42 149]]
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.75      0.78       231
           1       0.72      0.78      0.75       191

    accuracy                           0.76       422
   macro avg       0.76      0.76      0.76       422
weighted avg       0.77      0.76      0.76       422



In [ ]:
#   ---  Jupyter Notebook - Advanced Skin Cancer Classification  ---
   
#   Cell 1: Import Libraries

import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_curve,
    auc,
    accuracy_score,
)
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
    GlobalAveragePooling2D,
    Input,
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    EarlyStopping,
    ReduceLROnPlateau,
    TensorBoard,
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import (
    VGG16,
    InceptionV3,
    Xception,
    NASNetMobile,
)
from tensorflow.keras.regularizers import l2
import datetime

#   Cell 2: Define Helper Functions

def enhance_image(image_array):
    """Enhances image using CLAHE."""
    lab = cv2.cvtColor(image_array, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl, a, b))
    enhanced = cv2.cvtColor(merged, cv2.COLOR_LAB2RGB)
    return enhanced

def load_images_and_labels(dataset_path, img_size=(224, 224)):
    """Loads and preprocesses images and labels."""
    categories = ['benign', 'malignant']
    data = []
    labels = []

    for category in categories:
        folder_path = os.path.join(dataset_path, category)
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            image = cv2.imread(img_path)
            if image is not None:
                image = cv2.resize(image, img_size)
                image = enhance_image(image)
                data.append(image)
                labels.append(category)
            else:
                print(f"Error loading image: {img_path}")

    data = np.array(data, dtype="float32") / 255.0  #   Normalize
    labels = np.array(labels)
    encoder = LabelEncoder()
    labels = encoder.fit_transform(labels)  #   0 for benign, 1 for malignant
    labels = to_categorical(labels, num_classes=2)

    print(f"Loaded {len(data)} images.")
    return data, labels

def create_cnn_model(
    img_height, img_width, num_classes, l2_reg=0.0001, dropout_rate=0.5, filters=[32, 64, 128, 256], dense_units=256
):
    """Creates a customizable CNN model."""
    model = Sequential([
        Conv2D(
            filters[0],
            (3, 3),
            activation='relu',
            input_shape=(img_height, img_width, 3),
            kernel_regularizer=l2(l2_reg),
        ),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(
            filters[1],
            (3, 3),
            activation='relu',
            kernel_regularizer=l2(l2_reg),
        ),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(
            filters[2],
            (3, 3),
            activation='relu',
            kernel_regularizer=l2(l2_reg),
        ),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(
            filters[3],
            (3, 3),
            activation='relu',
            kernel_regularizer=l2(l2_reg),
        ),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(
            dense_units,
            activation='relu',
            kernel_regularizer=l2(l2_reg),
        ),
        Dropout(dropout_rate),
        Dense(
            num_classes,
            activation='softmax',
            kernel_regularizer=l2(l2_reg),
        ),
    ])

    model.compile(
        optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
    )
    model.summary()
    return model

def create_transfer_learning_model(
    base_model, img_height, img_width, num_classes, trainable_layers=-10
):
    """Creates a transfer learning model."""
    input_layer = Input(shape=(img_height, img_width, 3))
    x = base_model(input_layer, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    #   Freeze layers
    if trainable_layers > 0:
        for layer in base_model.layers[:trainable_layers]:
            layer.trainable = False
        for layer in base_model.layers[trainable_layers:]:
            layer.trainable = True
    else:
        for layer in base_model.layers:
            layer.trainable = False

    model.compile(
        optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
    )
    model.summary()
    return model

def train_model(
    model,
    train_data,
    train_labels,
    val_data,
    val_labels,
    use_augmentation=True,
    epochs=50,
    batch_size=32,
    checkpoint_path="skin_cancer_model.h5",
    class_weights=None,  #   For class imbalance
    experiment_name="experiment",
):
    """Trains the CNN model."""

    if use_augmentation:
        datagen = ImageDataGenerator(
            rotation_range=30,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='nearest',
        )
        datagen.fit(train_data)
        train_generator = datagen.flow(
            train_data, train_labels, batch_size=batch_size
        )
        training_data = train_generator
    else:
        training_data = (train_data, (train_labels))

    checkpoint = ModelCheckpoint(
        checkpoint_path,
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=False,
        mode='max',
        verbose=1,
    )
    early_stopping = EarlyStopping(
        monitor='val_loss', patience=15, restore_best_weights=True
    )
    lr_scheduler = ReduceLROnPlateau(
        monitor='val_accuracy', factor=0.5, patience=7, verbose=1, min_lr=1e-6
    )
    log_dir = "logs/fit/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

    history = model.fit(
        training_data,
        validation_data=(val_data, val_labels),
        epochs=epochs,
        callbacks=[checkpoint, early_stopping, lr_scheduler, tensorboard_callback],
        verbose=1,
        class_weight=class_weights,
    )
    return history

def evaluate_model(model, val_data, val_labels, model_name="model"):
    """Evaluates the trained model."""
    print(f"Evaluating {model_name}...")
    loss, accuracy = model.evaluate(val_data, val_labels, verbose=0)
    print(f"Validation Loss: {loss:.4f}")
    print(f"Validation Accuracy: {accuracy:.4f}")

    y_true = np.argmax(val_labels, axis=1)
    y_pred_prob = model.predict(val_data, verbose=0)
    y_pred = np.argmax(y_pred_prob, axis=1)

    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))

    fpr, tpr, _ = roc_curve(y_true, y_pred_prob[:, 1])
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve ({model_name})')
    plt.legend(loc="lower right")
    plt.show()

    return accuracy_score(y_true, y_pred)

#   Cell 3: Set Parameters

dataset_path = r"C:\Users\LLR User\Desktop\Coding\code\skin-cancer\Dataset"  #   <---  SET THIS!
img_height = 224
img_width = 224
num_classes = 2  #   Binary classification
batch_size = 32
epochs = 100  #   Increased epochs, EarlyStopping will handle it
use_data_augmentation = True  #   <---  SET THIS!

#   Hyperparameter Ranges (for basic experimentation)
l2_regs = [0.0001, 0.00001]
dropout_rates = [0.5, 0.4]
filter_sizes = [[32, 64, 128, 256], [64, 128, 256, 512]]
dense_units_sizes = [256, 512]

#   Cell 4: Load Data

data, labels = load_images_and_labels(dataset_path, (img_height, img_width))

#   Cell 5: Split Data

train_data, val_data, train_labels, val_labels = train_test_split(
    data, labels, test_size=0.2, random_state=42, stratify=np.argmax(labels, axis=1)
)

#   Cell 6: Calculate Class Weights (if needed)

from sklearn.utils import class_weight

if np.sum(train_labels[:, 0]) != np.sum(train_labels[:, 1]):  #   Check for imbalance
    class_weights = class_weight.compute_class_weight(
        'balanced', classes=np.unique(np.argmax(train_labels, axis=1)), y=np.argmax(train_labels, axis=1)
    )
    class_weights = {i: weight for i, weight in enumerate(class_weights)}
    print("Class Weights:", class_weights)
else:
    class_weights = None
    print("Data is balanced. Not using class weights.")

#   Cell 7: Model Training and Comparison

model_results = {}

#   1.   Custom CNN
print("---  Training Custom CNN  ---")
for l2_reg in l2_regs:
    for dropout_rate in dropout_rates:
        for filters in filter_sizes:
            for dense_units in dense_units_sizes:
                experiment_name = f"cnn_l2_{l2_reg}_drop_{dropout_rate}_filters_{filters[0]}_dense_{dense_units}"
                print(f"Experiment: {experiment_name}")
                cnn_model = create_cnn_model(
                    img_height,
                    img_width,
                    num_classes,
                    l2_reg=l2_reg,
                    dropout_rate=dropout_rate,
                    filters=filters,
                    dense_units=dense_units,
                )
                cnn_history = train_model(
                    cnn_model,
                    train_data,
                    train_labels,
                    val_data,
                    val_labels,
                    use_augmentation=use_data_augmentation,
                    epochs=epochs,
                    batch_size=batch_size,
                    checkpoint_path=f"cnn_{experiment_name}.h5",
                    class_weights=class_weights,
                    experiment_name=experiment_name
                )
                cnn_accuracy = evaluate_model(cnn_model, val_data, val_labels, model_name=experiment_name)
                model_results[experiment_name] = cnn_accuracy
                tf.keras.backend.clear_session()  #   Clear memory

#   2.   Transfer Learning Models
transfer_learning_models = {
    "VGG16": VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3)),
    "InceptionV3": InceptionV3(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3)),
    "Xception": Xception(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3)),
    "NASNetMobile": NASNetMobile(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3)),
}

for model_name, base_model in transfer_learning_models.items():
    print(f"---  Training {model_name}  ---")
    for trainable_layers in [-10, -50]:  #   Try different freezing strategies
        experiment_name = f"{model_name}_trainable_{trainable_layers}"
        print(f"Experiment: {experiment_name}")
        tl_model = create_transfer_learning_model(
            base_model, img_height, img_width, num_classes, trainable_layers=trainable_layers
        )
        tl_history = train_model(
            tl_model,
            train_data,
            train_labels,
            val_data,
            val_labels,
            use_augmentation=use_data_augmentation,
            epochs=epochs,
            batch_size=batch_size,
            checkpoint_path=f"{experiment_name}.h5",
            class_weights=class_weights,
            experiment_name=experiment_name
        )
        tl_accuracy = evaluate_model(tl_model, val_data, val_labels, model_name=experiment_name)
        model_results[experiment_name] = tl_accuracy
        tf.keras.backend.clear_session()

#   Cell 8: Model Comparison

print("\n---  Model Comparison  ---")
sorted_results = sorted(model_results.items(), key=lambda item: item[1], reverse=True)
for model_name, accuracy in sorted_results:
    print(f"{model_name}: Validation Accuracy = {accuracy:.4f}")

#   Cell 9: (Optional) Visualize Training History (Example)

def plot_history(history, model_name="model"):
    """Plots training history."""
    plt.plot(history.history['accuracy'], label=f'{model_name} train_accuracy')
    plt.plot(history.history['val_accuracy'], label=f'{model_name} val_accuracy')
    plt.plot(history.history['loss'], label=f'{model_name} train_loss')
    plt.plot(history.history['val_loss'], label=f'{model_name} val_loss')
    plt.legend()
    plt.title(f"Training History: {model_name}")

Loaded 2109 images.
Class Weights: {0: np.float64(0.9158523344191096), 1: np.float64(1.1011749347258486)}
---  Training Custom CNN  ---
Experiment: cnn_l2_0.0001_drop_0.5_filters_32_dense_256


c:\Users\LLR User\miniconda3\envs\tf-env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 24, 24, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36864)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     9,437,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,828,290 (37.49 MB)

 Trainable params: 9,827,330 (37.49 MB)

 Non-trainable params: 960 (3.75 KB)

c:\Users\LLR User\miniconda3\envs\tf-env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7209 - loss: 6.0426
Epoch 1: val_accuracy improved from -inf to 0.54739, saving model to cnn_cnn_l2_0.0001_drop_0.5_filters_32_dense_256.h5


53/53 ━━━━━━━━━━━━━━━━━━━━ 229s 4s/step - accuracy: 0.7208 - loss: 6.0331 - val_accuracy: 0.5474 - val_loss: 9.1959 - learning_rate: 0.0010
Epoch 2/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7363 - loss: 3.1584
Epoch 2: val_accuracy did not improve from 0.54739
53/53 ━━━━━━━━━━━━━━━━━━━━ 275s 5s/step - accuracy: 0.7364 - loss: 3.1435 - val_accuracy: 0.5474 - val_loss: 25.9322 - learning_rate: 0.0010
Epoch 3/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7724 - loss: 0.7863
Epoch 3: val_accuracy did not improve from 0.54739
53/53 ━━━━━━━━━━━━━━━━━━━━ 241s 4s/step - accuracy: 0.7722 - loss: 0.7865 - val_accuracy: 0.5474 - val_loss: 4.2304 - learning_rate: 0.0010
Epoch 4/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7504 - loss: 0.7168
Epoch 4: val_accuracy improved from 0.54739 to 0.72749, saving model to cnn_cnn_l2_0.0001_drop_0.5_filters_32_dense_256.h5
